In [144]:
import math

class Polygon:
    
    def __init__(self, n, R):
        if n < 3:
            raise ValueError('Polygon must have at least three sides.')
        self._n = n
        self._R = R
        
        self._interior_angle = None
        self._side_length = None
        self._apothem = None
        self._area = None
        self._perimeter = None

    @property
    def count_edges(self):
        return self._n

    @property
    def count_vertices(self):
        return self._n

    @property
    def circumradius(self):
        return self._R

    @property
    def interior_angle(self):
        if self._interior_angle is None:
            self._interior_angle = (self._n-2) * (180/self._n)
        return self._interior_angle
        

    @property
    def side_length(self):
        if self._side_length is None:
            self._side_length = 2*self._R*math.sin(math.pi/self._n)
        return self._side_length

    @property
    def apothem(self):
        if self._apothem is None:
            self._apothem = self._R*math.cos(math.pi/self._n)
        return self._apothem

    @property
    def area(self):
        if self._area is None:
            self._area = 0.5*self._n*self.side_length*self.apothem
        return self._area

    @property
    def perimeter(self):
        if self._perimeter is None:
            self._perimeter = self._n * self.side_length
        return self._perimeter

    def __repr__(self):
        return f'Polygon(edges(n) = {self.count_edges}, circumradius(R) = {self.circumradius})'
    
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.count_edges == other.count_edges and self.circumradius == other.circumradius
        else:
            return NotImplemented
            
    def __gt__(self, other):
        if isinstance(other, Polygon):
            return self.count_edges > other.count_edges
        else:
            return NotImplemented   

In [145]:
def test_polygon():
    rel_tol = 0.001
    abs_tol = 0.001
    
    try:
        p = Polygon(2, 10)
        assert False, ('Creating a Polygon with 2 sides: '
                        'Exception expected but not received.')
    except ValueError:
        pass

    n = 3
    R = 1
    p = Polygon(n, R)
    assert str(p) == f'Polygon(edges(n) = 3, circumradius(R) = 1)', f'actual: {str(p)}'
    assert p.count_vertices == n, (f'actual: {p.count_vertices}, '
                                   f' expected: {n}')
    assert p.count_edges == n
    assert p.circumradius == R
    assert p.interior_angle == 60

    n = 4
    R = 1
    p = Polygon(n, R)
    assert math.isclose(p.interior_angle, 90)
    assert math.isclose(p.area, 2.0,
                        rel_tol = rel_tol, 
                        abs_tol = abs_tol), f'actual: {p.area}, expected: 2.0'
    assert math.isclose(p.side_length, math.sqrt(2),
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    assert math.isclose(p.perimeter, 4 * math.sqrt(2),
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    assert math.isclose(p.apothem, 0.707,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    
    n = 6
    R = 2
    p = Polygon(n, R)
    assert math.isclose(p.interior_angle, 120)
    assert math.isclose(p.area, 10.3923,
                        rel_tol = rel_tol, 
                        abs_tol = abs_tol), f'actual: {p.area}, expected: 2.0'
    assert math.isclose(p.side_length, 2,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    assert math.isclose(p.perimeter, 12,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    assert math.isclose(p.apothem, 1.73205,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    
    n = 12
    R = 3
    p = Polygon(n, R)
    assert math.isclose(p.interior_angle, 150)
    assert math.isclose(p.area, 27,
                        rel_tol = rel_tol, 
                        abs_tol = abs_tol), f'actual: {p.area}, expected: 2.0'
    assert math.isclose(p.side_length, 1.55291,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    assert math.isclose(p.perimeter, 18.635,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)
    assert math.isclose(p.apothem, 2.89778,
                        rel_tol = rel_tol,
                        abs_tol = abs_tol)

    p1 = Polygon(3, 100)
    p2 = Polygon(10, 10)
    p3 = Polygon(15, 10)
    p4 = Polygon(15, 100)
    p5 = Polygon(15, 100)

    assert p2 > p1
    assert p2 < p3
    assert p3 != p4
    assert p1 != p4
    assert p4 == p5

In [146]:
test_polygon()

In [128]:
p = Polygon(1, 10)

ValueError: Polygon must have at least three sides.

In [129]:
class Polygons:

    def __init__(self, m, R):
        if m < 3:
            raise ValueError('m must be greater than 3.')
        self._m = m
        self._R = R
        self._length = None
#        self._polygons = [Polygon(i, R) for i in range(3, m+1)]
        self._max_efficiency_polygons = None

    def __len__(self):
        if self._length is None:
            self._length = self._m - 2
        return self._length
    
    def __iter__(self):
        return self.PolygonIterator(self._R, len(self))
        
    def __repr__(self):
        return f'Polygons(m={self._m}, R={self._R})'

    def __getitem__(self, i):
        if isinstance(i, slice):
            return self.PolygonIterator(self._R, len(range(i.start or 0, i.stop or len(self))), index=i.start or 0)
        else:
            return next(self.PolygonIterator(self._R, len(self), index=i))

    @property
    def max_efficiency_polygon(self):
        if self._max_efficiency_polygons is None:
            self._max_efficiency_polygons = sorted(iter(self),
                                                   key=lambda x: x.area / x.perimeter)[-1]
        return self._max_efficiency_polygons
    
    class PolygonIterator:
        
        def __init__(self, R, length, *, index=0):
            self._R = R
            self.length = index + length
            self.index = index
            
        def __iter__(self):
            return self
        
        def __next__(self):
            if self.index >= self.length:
                raise StopIteration
            else:
                result = Polygon(self.index+3, self._R)
                self.index += 1
                return result
                

In [130]:
polygons = Polygons(8, 1)

In [131]:
for p in polygons:
    print(p)

Polygon(edges(n) = 3, circumradius(R) = 1)
Polygon(edges(n) = 4, circumradius(R) = 1)
Polygon(edges(n) = 5, circumradius(R) = 1)
Polygon(edges(n) = 6, circumradius(R) = 1)
Polygon(edges(n) = 7, circumradius(R) = 1)
Polygon(edges(n) = 8, circumradius(R) = 1)


In [132]:
for p in polygons[2:5]:
    print(p)

Polygon(edges(n) = 5, circumradius(R) = 1)
Polygon(edges(n) = 6, circumradius(R) = 1)
Polygon(edges(n) = 7, circumradius(R) = 1)


In [133]:
for p in polygons[:3]:
    print(p)

Polygon(edges(n) = 3, circumradius(R) = 1)
Polygon(edges(n) = 4, circumradius(R) = 1)
Polygon(edges(n) = 5, circumradius(R) = 1)


In [134]:
for p in polygons[::-1]:
    print(p)

Polygon(edges(n) = 3, circumradius(R) = 1)
Polygon(edges(n) = 4, circumradius(R) = 1)
Polygon(edges(n) = 5, circumradius(R) = 1)
Polygon(edges(n) = 6, circumradius(R) = 1)
Polygon(edges(n) = 7, circumradius(R) = 1)
Polygon(edges(n) = 8, circumradius(R) = 1)


In [135]:
polygons[5]

Polygon(edges(n) = 8, circumradius(R) = 1)

In [136]:
len(polygons)

6

In [137]:
polygons

Polygons(m=8, R=1)

In [138]:
polygons.max_efficiency_polygon

Polygon(edges(n) = 8, circumradius(R) = 1)

In [139]:
polygons = Polygons(500, 1)

In [140]:
polygons.max_efficiency_polygon

Polygon(edges(n) = 500, circumradius(R) = 1)

In [141]:
p = polygons.max_efficiency_polygon

In [142]:
p.area

3.1415099708381518

In [143]:
assert math.isclose(p.area, math.pi, rel_tol=0.001, abs_tol=0.001)